In [ ]:
import numpy as np
import unyt

import scipy as sp
import scipy.interpolate

import matplotlib.pyplot as plt
import matplotlib as mpl

## Load data from [Schure 2009](https://doi.org/10.1051/0004-6361/200912495) Tables 2 and 4

In [ ]:
elemental_rates_data = np.genfromtxt("schure_table_4.txt",
                                skip_header=4,
                                dtype=float, delimiter='\t', names=True) 
elements = elemental_rates_data.dtype.names[1:]

log_temperature = elemental_rates_data["T"]

cooling_rates = unyt.unyt_array(np.empty( (log_temperature.size,len(elements)) ),"cm**3*erg/s")

for j,element in enumerate(elements):
    cooling_rates[:,j] = unyt.unyt_array(10**elemental_rates_data[element],"cm**3*erg/s")


schure_table2_data = np.loadtxt("schure_table_2.txt")

## Compute Cooling rates ($\Lambda_N$) from Schure Table 4 and eq. 3

In [ ]:
z1_log_cr = np.log10( cooling_rates.sum(axis=1) )

def compute_cr(n_over_n_sun):
    return (cooling_rates*n_over_n_sun).sum(axis=1)

z05_log_cr = np.log10( compute_cr(np.array( [1,1.0] + [1./2.,]*(len(elements)-2) ) ))
z03_log_cr = np.log10( compute_cr(np.array( [1,1] + [0.3,]*(len(elements)-2) ) ))

## Compute $\Lambda_{hd}$ from $n_e/n_H$ in Table 2 and eq. 1

In [ ]:
#ne_over_nh = sp.interpolate.CubicSpline(schure_table2_data[:,0],schure_table2_data[:,3])

start = np.where(schure_table2_data[:,0] == log_temperature[0])[0][0]
end = np.where(schure_table2_data[:,0] == log_temperature[-1])[0][0]
ne_over_nh = schure_table2_data[start:end+1,3]

In [ ]:
log_x_hd = np.log10(ne_over_nh)
schure_data = np.vstack((log_temperature,log_x_hd+z1_log_cr,log_x_hd+z05_log_cr,log_x_hd+z03_log_cr))

## Save the schure data with a thorough note

In [ ]:
header="""Cooling rates generated from Schure 2009 (doi.org/10.1051/0004-6361/200912495)
containing temperatures in the first column (in log10 K) and collisional ionisation
equilibrium (CIE) cooling rates (in log10 erg cm^3/s) for Z=1 solar metalicity, Z=0.5,
and Z=0.3 across the columns. Cooling rates are in the convention Lambda_hd from eq. 1,
where the proton ratio n_e/n_H is taken from table 2.
Lambda_N is computed from eq. 3. The cooling rate Lambda_N(X_i,T) from eq. 3 is contained
in table 4. n_i/n_i(solar) is taken to be 1.0 for all elements for Z=1 while for Z=0.5
and Z=0.5 n_i/n_i(solar) is set to 1 for H and He and set to 0.5 and 0.3 respectively for
all other elements. Made by Forrest Glines (forrestglines@gmail.com)
-----------------------------------------------------------------------------------------
log10 T [K] Z=1 log10 Lambda_N [erg cm^3/s] Z=0.5 log10 Lambda_N [erg cm^3/s] Z=0.3 log10 Lambda_N [erg cm^3/s]"""

In [ ]:
np.savetxt("updated_schure.cooling",schure_data.T,header=header,fmt=("%1.2f","%2.4f","%2.4f","%2.4f"))

## Load other cooling tables (See [Sutherland and Dopita 1993](https://ui.adsabs.harvard.edu/link_gateway/1993ApJS...88..253S/doi:10.1086/191823) )

In [ ]:
enzo_schure_data = np.loadtxt("enzo_schure.cooling") #Schure cooling table with enzo roots
#gnat_sternberg_data = np.loadtxt("gnat-sternberg.cooling")
#sutherland_dopita_data = np.loadtxt("cooling_data/sutherland_dopita.cooling") #SD table with PLUTO roots
sutherland_dopita_table6_data = np.loadtxt("sutherland_dopita_table_6.txt")

## Make some plots

In [ ]:
fig,ax = plt.subplots()

ax.plot(log_temperature,z1_log_cr,label="$\\Lambda_N$ Schure $Z_\\odot$")
ax.plot(log_temperature,z05_log_cr,label="$\\Lambda_N$ Schure $0.5 Z_\\odot$")
ax.plot(log_temperature,z03_log_cr,label="$\\Lambda_N$ Schure $0.3 Z_\\odot$")


ax.plot(log_temperature,log_x_hd+z1_log_cr,label="$\\Lambda_{hd}$ Schure $Z_\\odot$",linestyle="--")
ax.plot(log_temperature,log_x_hd+z05_log_cr,label="$\\Lambda_{hd}$ Schure $0.5 Z_\\odot$",linestyle="--")
ax.plot(log_temperature,log_x_hd+z03_log_cr,label="$\\Lambda_{hd}$ Schure $0.3 Z_\\odot$",linestyle="--")

#ax.plot(enzo_schure_data[:,0],enzo_schure_data[:,1],label="Legacy Schure $Z_\\odot$",linestyle="--")
#ax.plot(enzo_schure_data[:,0],enzo_schure_data[:,2],label="Legacy Schure $0.5 Z_\\odot$",linestyle="--")
#ax.plot(schure_table2_data[:,0],schure_table2_data[:,1],label="Schure Table 2 $Z=1$",linestyle="--")

#ax.plot(gnat_sternberg_data[:,0],gnat_sternberg_data[:,4],label="GS $Z=1$",linestyle=":")
#ax.plot(sutherland_dopita_data[:,0],sutherland_dopita_data[:,2],label="SD $Z=1$",linestyle="-")
#ax.plot(sutherland_dopita_data[:,0],sutherland_dopita_data[:,1],label="SD $Z=1/3$",linestyle=":")

#ax.plot(sutherland_dopita_table6_data[:,0],sutherland_dopita_table6_data[:,5],label="SD Table 6 $Z=1$",linestyle="--")

ax.legend(ncols=2)

#ax.set_xlim(log_temperature.min(),log_temperature.max())

ax.set_xlim(3.8,8.2)
ax.set_ylim(-23,-20)
#ax.set_ylim(-26,-20)

ax.set_ylabel("Cooling Rate -- $ \\log \\Lambda$ [ cm${}^{3}$ erg/s ]")
ax.set_xlabel("Temperature -- $ \\log T$ [ K ]")

ax.grid()

plt.show()


In [ ]:
fig,ax = plt.subplots()

ax.plot(log_temperature,log_x_hd+z1_log_cr,label="My $\\Lambda_{hd}$",linestyle="-")
ax.plot(log_temperature,schure_table2_data[start:end+1,2],label="Schure Table 2 $\\Lambda_{hd}$",linestyle=":")

ax.plot(log_temperature,z1_log_cr,label="My $\\Lambda_{N}$",linestyle="-")
ax.plot(log_temperature,enzo_schure_data[:,1],label="Enzo $\\Lambda_{N}$",linestyle="--")
ax.plot(log_temperature,schure_table2_data[start:end+1,2],label="Schure Table 1 $\\Lambda_{N}$",linestyle=":")

ax.legend()

#ax.set_xlim(log_temperature.min(),log_temperature.max())

ax.set_xlim(3.8,8.2)
ax.set_ylim(-23,-20)
#ax.set_ylim(-26,-20)

ax.set_ylabel("Cooling Rate -- $ \\log \\Lambda$ [ cm${}^{3}$ erg/s ]")
ax.set_xlabel("Temperature -- $ \\log T$ [ K ]")

ax.grid()

plt.show()


In [ ]:
fig,ax = plt.subplots()

ax.plot(log_temperature,
        (10**(log_x_hd+z1_log_cr) - 10**schure_table2_data[start:end+1,2])/10**(log_x_hd+z1_log_cr),
        label="$\\Lambda_{hd}$: Mine - Schure Table 2",linestyle="-")

ax.plot(log_temperature,
        (10**(z1_log_cr) - 10**schure_table2_data[start:end+1,1])/10**(z1_log_cr),
        label="$\\Lambda_{N}$: Mine - Schure Table 2",linestyle="--")
ax.plot(log_temperature,
        (10**(z1_log_cr) - 10**enzo_schure_data[:,1])/10**(z1_log_cr),
        label="$\\Lambda_{N}$: Mine - Enzo",linestyle="-")


ax.legend()

#ax.set_xlim(log_temperature.min(),log_temperature.max())

ax.set_xlim(3.8,8.2)
#ax.set_ylim(-23,-20)
#ax.set_ylim(-26,-20)

ax.set_ylabel("Relative difference in $\\Lambda$")
ax.set_xlabel("Temperature -- $ \\log T$ [ K ]")

ax.grid()

plt.show()